In [ ]:
from opensearchpy import OpenSearch
import pandas as pd

client = OpenSearch(
    hosts=[{'host': 'localhost', 'port': 9200}],
    http_auth=('admin', '4Z*lwtz,,2T:0TGu'),
    use_ssl=True,
    verify_certs=False  # Set to True if you have proper certificates
)


# Query OpenSearch to fetch all document IDs
index_name = "tprocanswers"
query = {
    "_source": False,
    "query": {"match_all": {}}
}

# Scroll API for fetching large datasets
result = client.search(index=index_name, body=query, scroll="2m", size=1000)
scroll_id = result["_scroll_id"]
hits = result["hits"]["hits"]

# Collect IDs
ids = [hit["_id"] for hit in hits]

# Scroll through all results
while len(hits) > 0:
    result = client.scroll(scroll_id=scroll_id, scroll="2m")
    scroll_id = result["_scroll_id"]
    hits = result["hits"]["hits"]
    ids.extend(hit["_id"] for hit in hits)

# Split data if it exceeds Excel's row limit
max_rows = 1000000
file_count = 1

for i in range(0, len(ids), max_rows):
    # Create a DataFrame for the current chunk
    chunk = ids[i:i + max_rows]
    df = pd.DataFrame(chunk, columns=["tcno"])
    
    # Save to an Excel file
    file_name = f"document_ids_part{file_count}.xlsx"
    df.to_excel(file_name, index=False)
    print(f"Saved: {file_name}")
    file_count += 1

print("All document IDs have been saved in multiple Excel files.")


In [1]:

import sys, ast
sys.path.append(r'/data/imageExtraction/')
sys.path.append(r'/data/imageExtraction/imgEnv/')

In [2]:
print(sys.path)

['/usr/lib64/python311.zip', '/usr/lib64/python3.11', '/usr/lib64/python3.11/lib-dynload', '', '/home/rajeev/.local/lib/python3.11/site-packages', '/usr/local/lib64/python3.11/site-packages', '/usr/local/lib/python3.11/site-packages', '/usr/lib64/python3.11/site-packages', '/usr/lib/python3.11/site-packages', '/data/imageExtraction/', '/data/imageExtraction/imgEnv/']


In [3]:
# from googleapiclient.http import MediaIoBaseUpload, MediaIoBaseDownload
import imageAPI_live as imgProcessingCode

In [4]:
import os, torch
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import re
from sentence_transformers import util
import warnings

/home/rajeev/.local/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [5]:

import torch, re, warnings, io, time, ast, logging
import numpy as np
import pandas as pd
from googleapiclient.discovery import build
from google.oauth2 import service_account
from apiclient.http import MediaFileUpload, MediaIoBaseDownload
from googleapiclient.http import MediaIoBaseUpload, MediaIoBaseDownload
from deep_translator import GoogleTranslator
from langdetect import detect
import concurrent.futures

In [2]:
import sys
sys.path.append(r'/data/imageExtraction/')
sys.path.append(r'/data/imageExtraction/imgEnv/')

try:
    from deep_translator import GoogleTranslator
except ModuleNotFoundError:
    print("deep_translator module not found. Please check installation.")


In [1]:
import psycopg2
from psycopg2 import OperationalError

# Attempt to connect to the database
try:
    connection = psycopg2.connect(
            user="boq",
            password="boq",
            host="localhost",
            port="5432",
            database="tbl_docboq"
        )
    # If connected, print success message
    print("Connection successful")
except OperationalError as e:
    print(f"Error: Unable to connect to the database: {e}")
finally:
    # Closing the connection
    if connection:
        connection.close()
        print("Connection closed")


Connection successful
Connection closed


In [ ]:
outFile=r'C:\Users\hetvi.solanki\Downloads\77153810.txt'
with open(outFile, 'r', encoding='utf-8') as f:
    essay = f.read()

 
import re

# Splitting the essay on '.', '?', and '!'
single_sentences_list = re.split(r'(?<=[.?!])\s+', essay)
print (f"{len(single_sentences_list)} senteneces were found")

 
sentences = [{'sentence': x, 'index' : i} for i, x in enumerate(single_sentences_list)]
sentences[:3]

 
def combine_sentences(sentences, buffer_size=1):
    # Go through each sentence dict
    for i in range(len(sentences)):

        # Create a string that will hold the sentences which are joined
        combined_sentence = ''

        # Add sentences before the current one, based on the buffer size.
        for j in range(i - buffer_size, i):
            # Check if the index j is not negative (to avoid index out of range like on the first one)
            if j >= 0:
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += sentences[j]['sentence'] + ' '

        # Add the current sentence
        combined_sentence += sentences[i]['sentence']

        # Add sentences after the current one, based on the buffer size
        for j in range(i + 1, i + 1 + buffer_size):
            # Check if the index j is within the range of the sentences list
            if j < len(sentences):
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += ' ' + sentences[j]['sentence']

        # Then add the whole thing to your dict
        # Store the combined sentence in the current sentence dict
        sentences[i]['combined_sentence'] = combined_sentence

    return sentences

sentences = combine_sentences(sentences)

 
sentences[:3]

 
from langchain.embeddings import OpenAIEmbeddings
oaiembeds = OpenAIEmbeddings()

 
embeddings = oaiembeds.embed_documents([x['combined_sentence'] for x in sentences])

 
for i, sentence in enumerate(sentences):
    sentence['combined_sentence_embedding'] = embeddings[i]

 
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_distances(sentences):
    distances = []
    for i in range(len(sentences) - 1):
        embedding_current = sentences[i]['combined_sentence_embedding']
        embedding_next = sentences[i + 1]['combined_sentence_embedding']
        
        # Calculate cosine similarity
        similarity = cosine_similarity([embedding_current], [embedding_next])[0][0]
        
        # Convert to cosine distance
        distance = 1 - similarity

        # Append cosine distance to the list
        distances.append(distance)

        # Store distance in the dictionary
        sentences[i]['distance_to_next'] = distance

    # Optionally handle the last sentence
    # sentences[-1]['distance_to_next'] = None  # or a default value

    return distances, sentences

 
distances, sentences = calculate_cosine_distances(sentences)

import matplotlib.pyplot as plt

plt.plot(distances);

 
import numpy as np

plt.plot(distances);

y_upper_bound = .2
plt.ylim(0, y_upper_bound)
plt.xlim(0, len(distances))

# We need to get the distance threshold that we'll consider an outlier
# We'll use numpy .percentile() for this
breakpoint_percentile_threshold = 95 #95
breakpoint_distance_threshold = np.percentile(distances, breakpoint_percentile_threshold) # If you want more chunks, lower the percentile cutoff
plt.axhline(y=breakpoint_distance_threshold, color='r', linestyle='-');

# Then we'll see how many distances are actually above this one
num_distances_above_theshold = len([x for x in distances if x > breakpoint_distance_threshold]) # The amount of distances above your threshold
plt.text(x=(len(distances)*.01), y=y_upper_bound/50, s=f"{num_distances_above_theshold + 1} Chunks");

# Then we'll get the index of the distances that are above the threshold. This will tell us where we should split our text
indices_above_thresh = [i for i, x in enumerate(distances) if x > breakpoint_distance_threshold] # The indices of those breakpoints on your list

# Start of the shading and text
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
for i, breakpoint_index in enumerate(indices_above_thresh):
    start_index = 0 if i == 0 else indices_above_thresh[i - 1]
    end_index = breakpoint_index if i < len(indices_above_thresh) - 1 else len(distances)

    plt.axvspan(start_index, end_index, facecolor=colors[i % len(colors)], alpha=0.25)
    plt.text(x=np.average([start_index, end_index]),
             y=breakpoint_distance_threshold + (y_upper_bound)/ 20,
             s=f"Chunk #{i}", horizontalalignment='center',
             rotation='vertical')

# # Additional step to shade from the last breakpoint to the end of the dataset
if indices_above_thresh:
    last_breakpoint = indices_above_thresh[-1]
    if last_breakpoint < len(distances):
        plt.axvspan(last_breakpoint, len(distances), facecolor=colors[len(indices_above_thresh) % len(colors)], alpha=0.25)
        plt.text(x=np.average([last_breakpoint, len(distances)]),
                 y=breakpoint_distance_threshold + (y_upper_bound)/ 20,
                 s=f"Chunk #{i+1}",
                 rotation='vertical')

plt.title("PG Essay Chunks Based On Embedding Breakpoints")
plt.xlabel("Index of sentences in essay (Sentence Position)")
plt.ylabel("Cosine distance between sequential sentences")
plt.show()

# Initialize the start index
start_index = 0

# Create a list to hold the grouped sentences
chunks = []

# Iterate through the breakpoints to slice the sentences
for index in indices_above_thresh:
    # The end index is the current breakpoint
    end_index = index

    # Slice the sentence_dicts from the current start index to the end index
    group = sentences[start_index:end_index + 1]
    combined_text = ' '.join([d['sentence'] for d in group])
    chunks.append(combined_text)
    
    # Update the start index for the next group
    start_index = index + 1

# The last group, if any sentences remain
if start_index < len(sentences):    
    combined_text = ' '.join([d['sentence'] for d in sentences[start_index:]])
    chunks.append(combined_text)

# grouped_sentences now contains the chunked sentences

 
# for i, chunk in enumerate(chunks[:20]):
#     buffer = 200
    
#     print(f"Chunk #{i}")
#     print(chunk.strip())
#     # print ("...")
#     # print (chunk[-buffer:].strip())
#     print("\n")

   
# ## Semantic Clustering

 
import json
from sentence_transformers import SentenceTransformer
import numpy as np
from collections import defaultdict
import faiss
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Load the model and initialize labels
model = SentenceTransformer('all-MiniLM-L6-v2')
labels = ["Important Date", "Eligibility or Prequalification Criteria", "Technical Requirements", "Contact Details"]
label_embeddings = model.encode(labels)  # Get embeddings for labels

# Initialize FAISS index with label embeddings
dimension = label_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)  # L2 distance index
faiss_index.add(label_embeddings)  # Add label embeddings to FAISS index

def segment_text_with_faiss_label_assignment(semantic_chunks):
    labeled_segments = defaultdict(list)
    
    for chunk in semantic_chunks:
        if chunk.strip():  # Skip empty paragraphs
            paragraph_embedding = model.encode(chunk).reshape(1, -1)
            _, closest_label_index = faiss_index.search(paragraph_embedding, 1)
            closest_label = labels[closest_label_index[0][0]]
            labeled_segments[closest_label].append(chunk)
     
    return labeled_segments

RELEVANCE_THRESHOLD = -0.7  # Adjust based on experimentation

def segment_text_with_faiss_label_assignment(semantic_chunks, threshold=RELEVANCE_THRESHOLD):
    labeled_segments = defaultdict(list)
    irrelevant_segments = []  # For storing irrelevant chunks

    for chunk in semantic_chunks:
        if chunk.strip():  # Skip empty paragraphs
            paragraph_embedding = model.encode(chunk).reshape(1, -1)
            
            # Get similarity scores with all labels
            distances, label_indices = faiss_index.search(paragraph_embedding, len(labels))
            similarities = 1 - distances  # Convert distances to cosine similarity
            # for i, sim in enumerate(similarities[0]):
            #     #print(f"Label: {labels[i]}, Similarity: {sim}")
            # Assign to multiple labels if similarity exceeds threshold
            assigned_labels = [labels[i] for i, sim in enumerate(similarities[0]) if sim >= threshold]

            if assigned_labels:
                print("Assigned labels : ")
                print(assigned_labels)
                # Add chunk to all relevant labels
                for label in assigned_labels:
                    labeled_segments[label].append(chunk)
            else:
                # If no label has sufficient similarity, mark it as irrelevant
                labeled_segments["Other"].append(chunk)
    
    for label in labels:
        print(label , len(labeled_segments[label]))
    # Add the irrelevant chunks to an "Other" category
    # labeled_segments["Other"] = irrelevant_segments
    return labeled_segments
 
segmented_result = segment_text_with_faiss_label_assignment(chunks)
#print(" == Segmented Results == ")
#print(segmented_result)
# Save the result to JSON
out_file_path = r'C:\Users\hetvi.solanki\Desktop\AIProjects\ragllm\ragTechniques\RAG_Techniques\data\out.json'
with open(out_file_path, 'w', encoding='utf-8') as out_file:
    json.dump(segmented_result, out_file, indent=4, ensure_ascii=False)

 
# Prepare data for visualization
colors = ['red', 'blue', 'green', 'yellow']  # Define colors for each cluster
scatter_points = []
labels_for_plot = []

for label_index, label in enumerate(labels):
    paragraphs = segmented_result[label]
    #print("--para--", label)
    #print(paragraphs)
    paragraph_embeddings = model.encode(paragraphs)
    scatter_points.append(paragraph_embeddings)
    labels_for_plot.extend([label] * len(paragraphs))

# Concatenate all paragraph embeddings for plotting
all_embeddings = np.vstack(scatter_points)

# Use KMeans to cluster embeddings
n_clusters = len(labels)  # Number of clusters based on labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(all_embeddings)
cluster_labels = kmeans.labels_

# Reduce dimensions for visualization
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(all_embeddings)
 
# Create scatter plot
plt.figure(figsize=(10, 6))
# labeled_final_chunk=[]
# #print(len(chunks))
for i, label in enumerate(labels):
    indices = np.where(cluster_labels == i)[0]  # Get indices of paragraphs in this cluster
    # para=""
    # #print(indices)
    # for index in indices:
    #     para += chunks[index]
    # labeled_final_chunk.append((label, para))
    ##print(label, " ", colors[i])
    ##print(indices)
    plt.scatter(reduced_embeddings[indices, 0], reduced_embeddings[indices, 1], 
                label=label, color=colors[i], alpha=0.7)

plt.title('Text Segmentation Clusters with K-Means')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend()
plt.grid()
plt.show()

 
##print("labeled_final_chunk : ", labeled_final_chunk)

 
def search_chunk(label_name, labeled_final_chunk):
    for label, para in labeled_final_chunk:
            if label == label_name:  # Check if the label matches
                return para 

eligibility_criteria_chunk = segmented_result["Eligibility or Prequalification Criteria"]
# ## LLM 

from openai import OpenAI
import concurrent.futures
client = OpenAI(
    base_url="http://129.154.248.128:8000/v1",
    api_key="token-abc123",
)
model=client.models.list().data[0].id

# MAX_TOKENS = 8000

# def split_chunk_for_llm(chunk, max_tokens=MAX_TOKENS):
#     # Assuming 1 token ~ 4 characters on average
#     avg_chars_per_token = 4
#     max_chars = max_tokens * avg_chars_per_token
    
#     if len(chunk) > max_chars:
#         # Split chun...